In [1]:
import numpy as np
import random
import statistics


# Card Values

In [2]:
ace_value=-1
high_value=-1
mid_value=0
low_value= 1

In [3]:
#Make sure expected value is 0

ace_value* 1/13 + high_value * 4/13 + mid_value* 3/13 + low_value*5/13

0.0

# Initial Parameters

In [4]:
num_decks=4
#avg cards dealt per hand
cards_per_hand= 7
num_hands=8
num_sims=100_000
init_edge= -0.5/100
#Change in edge by change in effective count
delta_edge = 0.5/100
min_bet=1
max_bet=100
kelly_roll=500_000
min_edge= min_bet/ kelly_roll
max_edge= max_bet/ kelly_roll

# Helper Functions

In [5]:
def deck_creator(n):
    return ['A']*4*n + ['H']*16*n + ['M']*12*n + ['L']*20*n

In [6]:
def hand_sim(deck, num_cards):
    cards=[]
    while len(cards)<num_cards:
        l= len(deck)
        i= random.randint(0,l-1)
        cards.append(deck[i])
        del deck[i]
    return (deck, cards)

In [7]:
def init_counter(deck):
    count=0
    for c in deck:
        if c=='A':
            count-= ace_value
        
        if c=='H':
            count-= high_value
            
        if c=='M':
            count-=mid_value
            
        if c=='L':
            count-=low_value
            
    return count

In [8]:
def one_count_sim(deck, cards_per_hand, num_hands):
    start_count= init_counter(deck)*52/len(deck)
    counts=[start_count]
    d=deck
    for i in range(num_hands - 1):
        hand= hand_sim(d, cards_per_hand)
        d= hand[0]
        counts.append(-52 * init_counter(hand[-1])/ len(d))
        
        
    return counts
        
    

In [9]:
d= deck_creator(1)

print(init_counter(d))


one_count_sim(d, 5,5)

0


[0.0, 2.2127659574468086, -3.7142857142857144, 1.4054054054054055, 0.0]

In [10]:
def n_count_sims(n_sims, init_deck, cards_per_hand, num_hands):
    counts=[]
    for i in range(n_sims):
        d= init_deck.copy()
        sub_count= one_count_sim(d, cards_per_hand, num_hands)
        counts+=sub_count
    return counts

In [11]:
results= n_count_sims(num_sims, deck_creator(num_decks), cards_per_hand, num_hands)

In [12]:
len(results)

800000

In [13]:
def edge_amount(count):
    return init_edge + delta_edge*count

In [ ]:
exp_pnl=0
edges=[]
for r in results:
    edge= edge_amount(r)
    if edge<min_edge:
        e= min_bet*edge
        exp_pnl+= e
        edges.append(e)
        
    elif edge>max_edge:
        e= max_bet*edge
        exp_pnl += e
        edges.append(e)
        
    else:
        e=kelly_roll*(edge**2)
        exp_pnl += e
        edges.append(e)
        

# Results

In [14]:

#pnl_amount
exp_pnl

2618.0867501114903

In [15]:
#sds above 0
sum(edges)/ (np.sqrt(len(edges)) *statistics.stdev(edges))

64.9911451436244